http://karpathy.github.io/neuralnets/

## circuits with one gate

In [1]:
def forwardMultiplyGate(x,y):
    return x * y

x = -2
y = 3

In [2]:
import numpy as np
tweak = 0.01

In [4]:
best_out = - np.inf
best_x = x 
best_y = y

In [5]:
# random local search

for i in xrange(100):
    x_try = x + tweak * (np.random.normal()*2 - 1)
    y_try = y + tweak * (np.random.normal()*2 - 1)
    out = forwardMultiplyGate(x_try, y_try)
    if out > best_out:
        best_out = out
        best_x = x_try
        best_y = y_try

In [6]:
print best_x,best_y,best_out

-1.99351370814 2.95278236817 -5.88641212812


In [11]:
# numerical gradient
# intuition: force x to be larger, force y to be lower, to make output larger than -6

x = -2
y = 3
out = forwardMultiplyGate(x,y)
h = 0.0001 # small enough, works fine in most cases

xph = x + h
out2 = forwardMultiplyGate(xph,y)
x_derivative = (out2 - out)/h
x_derivative

3.00000000000189

In [12]:
yph = y + h
out3 = forwardMultiplyGate(x, yph)
y_derivative = (out3 - out)/h 
y_derivative

-2.0000000000042206

In [13]:
step_size = 0.01 # bigger steps not always better
out = forwardMultiplyGate(x,y)
x = x + step_size * x_derivative
y = y + step_size * y_derivative
out_new = forwardMultiplyGate(x,y)
out_new # output increase

-5.87059999999986

In [14]:
# analytic gradient

x = -2
y = 3
out = forwardMultiplyGate(x,y)
x_gradient = y # f(x,y)=xy, x's gradient is y
y_gradient = x # y's gradient is x

In [15]:
step_size = 0.01
x += step_size * x_gradient
y += step_size * y_gradient
out_new = forwardMultiplyGate(x,y)
out_new

-5.8706

实际运用中, 都会计算 analytic gradient, 但用numerical gradient 来验证. 因为 numerical 的方法虽然简单但计算昂贵, analytic 虽然有 bugs 但会很高效

## circuits with multiple gates

In [16]:
def forwardMultiplyGate(a,b):
    return a * b
def forwardAddGate(a,b):
    return a + b

def forwardCircuit(x,y,z):
    q = forwardAddGate(x,y) # q = x + y
    f = forwardMultiplyGate(q,z) # f = q*z
    return f

x = -2
y = 5
z = -4
forwardCircuit(x,y,z)

-12

### backpropagation
Chain rule

$\frac{\partial f(q,z)}{\partial x} = \frac{\partial q(x,y)}{\partial x} \frac{\partial f(q,z)}{\partial q}$

In [17]:
x = -2
y = 5
z = -4
q = forwardAddGate(x,y)
f = forwardMultiplyGate(q,z)

derivative_f_wrt_z = q
derivative_f_wrt_q = z

derivative_q_wrt_x = 1.0
derivative_q_wrt_y = 1.0

derivative_f_wrt_x = derivative_q_wrt_x * derivative_f_wrt_q
derivative_f_wrt_y = derivative_q_wrt_y * derivative_f_wrt_q

In [18]:
gradient_f_wrt_xyz = [derivative_f_wrt_x, derivative_f_wrt_y, derivative_f_wrt_z]

In [19]:
step_size = 0.01
x = x + step_size * derivative_f_wrt_x
y = y + step_size * derivative_f_wrt_y
z = z + step_size * derivative_f_wrt_z

q = forwardAddGate(x,y)
q

2.92

In [20]:
f = forwardMultiplyGate(q,z)
f # up from -12

-11.5924

### backward flow

In [23]:
# numerical gradient check

x = -2
y = 5
z = -4

h = 0.0001
x_derivative = (forwardCircuit(x+h,y,z) - forwardCircuit(x,y,z)) / h
y_derivative = (forwardCircuit(x,y+h,z) - forwardCircuit(x,y,z)) / h
z_derivative = (forwardCircuit(x,y,z+h) - forwardCircuit(x,y,z)) / h
print x_derivative, y_derivative, z_derivative

-3.99999999999 -3.99999999999 3.00000000001


### example: single neuron
 f(x,y,a,b,c)= σ(ax+by+c)

sigmoid function - output 0 to 1

$\frac{\partial \sigma(x)}{\partial x}= \sigma(x)(1-\sigma(x))$

In [25]:
class Unit:
    def __init__(this, value,grad):
        this.value = value
        this.grad = grad

In [33]:
class multiplyGate:
    def forward(this, u0, u1):
        # store pointers to input Units u0 and u1 and output unit utop
        this.u0 = u0
        this.u1 = u1
        this.utop = Unit(u0.value*u1.value, 0.0)
        return this.utop
    
    def backward(this):
        # take the gradient in output unit and chain it with the
        # local gradients, which we derived for multiply gate before
        # then write those gradients to those Units.
        this.u0.grad += this.u1.value * this.utop.grad
        this.u1.grad += this.u0.value * this.utop.grad

In [54]:
class addGate:
    def forward(this, u0, u1):
        this.u0 = u0
        this.u1 = u1
        this.utop = Unit(u0.value + u1.value, 0.0)
        return this.utop
    def backward(this):
        this.u0.grad += 1 * this.utop.grad
        this.u1.grad += 1 * this.utop.grad

In [55]:
def sig(x):
    return 1 / (1 + np.exp(-x))

class sigmoidGate:
    
    def forward(this, u0):
        this.u0 = u0
        this.utop = Unit(sig(this.u0.value), 0.0)
        return this.utop
    def backward(this):
        s = sig(this.u0.value)
        this.u0.grad += (s*(1-s))*this.utop.grad


In [56]:
# inputs
a = Unit(1.0,0.0)
b = Unit(2.0,0.0)
c = Unit(-3.0,0.0)
x = Unit(-1.0,0.0)
y = Unit(3.0,0.0)

# gates
mulg0 = multiplyGate()
mulg1 = multiplyGate()
addg0 = addGate()
addg1 = addGate()
sg0 = sigmoidGate()

In [57]:
# forward pass forwardNeuron
def forwardNeuron():
    ax = mulg0.forward(a,x)
    by = mulg1.forward(b,y)
    axpby = addg0.forward(ax,by)
    axpbypc = addg1.forward(axpby,c)
    s = sg0.forward(axpbypc)
    return s
s = forwardNeuron()
print s.value

0.880797077978


In [58]:
# backward 
s.grad = 1.0 # set to be 1, if not, all gradient wil be 0
sg0.backward()   # writes gradient into axpbypc
addg1.backward() # writes gradients into axpby and c
addg0.backward() # writes gradients into ax and by
mulg1.backward() # writes gradients into b and y
mulg0.backward() # writes gradients into a and x

In [59]:
step_size = 0.01

a.value += step_size * a.grad
b.value += step_size * b.grad
c.value += step_size * c.grad
x.value += step_size * x.grad
y.value += step_size * y.grad

s = forwardNeuron()
print s.value # 0.88255 is higher than 0.88080

0.882550181622


In [60]:
# verify implemented backpropagation

def forwardCircuitFast(a,b,c,x,y):
    return 1/(1 + np.exp( - (a*x + b*y + c)))

a = 1
b = 2
c = -3
x = -1
y = 3

h = 0.0001
a_grad = (forwardCircuitFast(a+h,b,c,x,y) - forwardCircuitFast(a,b,c,x,y))/h
b_grad = (forwardCircuitFast(a,b+h,c,x,y) - forwardCircuitFast(a,b,c,x,y))/h
c_grad = (forwardCircuitFast(a,b,c+h,x,y) - forwardCircuitFast(a,b,c,x,y))/h
x_grad = (forwardCircuitFast(a,b,c,x+h,y) - forwardCircuitFast(a,b,c,x,y))/h
y_grad = (forwardCircuitFast(a,b,c,x,y+h) - forwardCircuitFast(a,b,c,x,y))/h

In [61]:
print [a_grad,b_grad,c_grad,x_grad,y_grad]

[-0.10499758359205913, 0.31494477483517969, 0.10498958734506125, 0.10498958734506125, 0.2099711788272618]


> All you have to do is write small gates that compute local, simple derivatives w.r.t their inputs, wire it up in a graph, do a forward pass to compute the output value and then a backward pass that chains the gradients all the way to the input.

In [ ]:
# multiply gate - swither
x = a*b
da = b*dx
db = a*dx

# add gate
x = a + b
da = 1 * dx
db = 1 * dx

# x = a + b + c compute in two steps
q = a + b
x = q + c
# backward pass
dc = 1 * dx
dq = 1 * dx
da = 1 * dq
db = 1 * dq
# -->
x = a + b + c
da = 1 * dx
db = 1 * dx
dc = 1 * dx

# combining gates
x = a * b + c
da = b * dx
db = a * dx
dc = 1 * dx 

# neuron case
q = a*x + b*y + c
f = sig(q)
df = 1
dq = (f*(1-f))*df
da = x * dq
dx = a * dq
dy = b * dq
db = y * dq
dc = 1 * dq


chapter 1 done

In [62]:
# binary classification
'''
vector -> label
---------------
[1.2, 0.7] -> +1
[-0.3, 0.5] -> -1
[-3, -1] -> +1
[0.1, 1.0] -> -1
[3.0, 1.1] -> -1
[2.1, -3] -> +1
'''

# learn support vector machine

#A circuit: it takes 5 Units (x,y,a,b,c) and outputs a single Unit
# It can also compute the gradient w.r.t. its inputs
class Circuit:
    def __init__(this): # gates
        this.mulg0 = multiplyGate()
        this.mulg1 = multiplyGate()
        this.addg0 = addGate()
        this.addg1 = addGate()
    def forward(this,x,y,a,b,c):
        this.ax = this.mulg0.forward(a,x)
        this.by = this.mulg1.forward(b,y)
        this.axpby = this.addg0.forward(this.ax, this.by)
        this.axpbypc = this.addg1.forward(this.axpby, c)
        return this.axpbypc
    def backward(this,gradient_top):
        this.axpbypc.grad = gradient_top
        this.addg1.backward()
        this.addg0.backward()
        this.mulg1.backward()
        this.mulg0.backward()

In [69]:
# SVM class
class SVM:
    def __init__(this):
        this.a = Unit(1.0,0.0)
        this.b = Unit(-2.0,0.0)
        this.c = Unit(-1.0,0.0)
        this.circuit = Circuit()
    def forward(this,x,y):
        this.unit_out = this.circuit.forward(x,y, this.a, this.b, this.c)
        return this.unit_out
    def backward(this,label): # label is +1 or -1
        this.a.grad = 0.0
        this.b.grad = 0.0
        this.c.grad = 0.0
        pull = 0.0
        if label==1 and this.unit_out.value<1:
            pull = 1
        if label==-1 and this.unit_out.value>-1:
            pull = -1
        this.circuit.backward(pull)
        
        this.a.grad += -this.a.value
        this.b.grad += -this.b.value
    def learnFrom(this,x,y,label):
        this.forward(x,y)
        this.backward(label)
        this.parameterUpdate()
    def parameterUpdate(this):
        step_size = 0.01
        this.a.value += step_size * this.a.grad
        this.b.value += step_size * this.b.grad
        this.c.value += step_size * this.c.grad

In [70]:
data = [[1.2,0.7],[-0.3,-0.5],[3.0,0.1],[-0.1,-1.0],[-1.0,1.1],[2.1,-3]]
labels = [1,-1,1,-1,-1,1]

In [82]:
# train SVM with SGD

svm = SVM()

#evaluate
def evalTrainingAccuracy():
    num_correct = 0
    for i in range(len(data)):
        x = Unit(data[i][0],0.0)
        y = Unit(data[i][1],0.0)
        true_label = labels[i]
        if svm.forward(x,y).value > 0:
            predicted_label = 1
        else:
            predicted_label = -1
        if predicted_label == true_label:
            num_correct += 1
    return float(num_correct)/len(data)

In [88]:
# learning loop ???

for iteration in range(400):
    i = int(np.floor(np.random.uniform()*len(data)))
    x = Unit(data[i][0],0.0)
    y = Unit(data[i][1],0.0)
    label = labels[i]
    svm.learnFrom(x,y,label)
    if iteration%25 == 0:
        print "training accuracy at iteration",iteration,":",evalTrainingAccuracy()

training accuracy at iteration 0 : 0.833333333333
training accuracy at iteration 25 : 0.833333333333
training accuracy at iteration 50 : 0.833333333333
training accuracy at iteration 75 : 0.833333333333
training accuracy at iteration 100 : 0.833333333333
training accuracy at iteration 125 : 0.833333333333
training accuracy at iteration 150 : 0.833333333333
training accuracy at iteration 175 : 0.833333333333
training accuracy at iteration 200 : 0.833333333333
training accuracy at iteration 225 : 1.0
training accuracy at iteration 250 : 0.833333333333
training accuracy at iteration 275 : 0.833333333333
training accuracy at iteration 300 : 0.833333333333
training accuracy at iteration 325 : 1.0
training accuracy at iteration 350 : 0.833333333333
training accuracy at iteration 375 : 0.833333333333


In [90]:
# train SVM simpler way
a = 1
b = -2
c = -1

for iteration in range(400):
    i = int(np.floor(np.random.uniform()*len(data)))
    x = data[i][0]
    y = data[i][1]
    label = labels[i]
    score = a*x+b*y+c
    pull = 0.0
    if label ==1 and score <1:
        pull = 1
    if label == -1 and score>-1:
        pull = -1
    step_size = 0.01
    a += step_size*(x*pull-a)
    b += step_size*(y*pull-b)
    c += step_size*(1*pull)

In [ ]:
# generailizing SVM into Neural Nets
# 2-layer neural nets does binary classification
n1 = max(0, a1*x + b1*y + c1)
n2 = max(0, a2*x + b2*y + c2)
n3 = max(0, a3*x + b3*y + c3)
score = a4*n1 + b4*n2 + c4*n3 + d4

# 3 hidden neurons, n1,n2,n3, Rectified Linear Unit (ReLU) non-linearity on each hidden neuron

In [ ]:
# every hidden neuron is a classifier, build extra classifier on top of them 
a1 = np.random.uniform()-0.5 #a random number between -0.5 and 0.5
for iteration in range(400):
    i = int(np.floor(np.random.uniform()*len(data)))
    x = data[i][0]
    y = data[i][1]
    label = labels[i]
    n1 = max(0, a1*x + b1*y + c1)
    n2 = max(0, a2*x + b2*y + c2)
    n3 = max(0, a3*x + b3*y + c3)
    score = a4*n1 + b4*n2 + c4*n3 + d4
    
    pull = 0.0
    if label ==1 and score <1:
        pull = 1
    if label == -1 and score>-1:
        pull = -1
    # backprop through the last "score" neuron     
    dscore = pull
    da4 = n1 * dscore
    dna = a4 * dscore
    db4 = n2 * dscore
    dn2 = b4 * dscore
    dc4 = n3 * dscore
    dn3 = c4 * dscore
    dd4 = 1.0 * dscore
    # backprop the ReLU non-linearities, in place
    # i.e. just set gradients to zero if the neurons did not "fire"
    if n3 == 0:
        dn3 = 0
    if n2 == 0:
        dn2 = 0
    if n1 == 0:
        dn1 = 0
    
    # backprop tp neuron 1
    da1 = x * dn1
    db1 = y * dn1
    dc1 = 1.0 * dn1
    
    # backprop to neuron 2
    da2 = x * dn2
    db2 = y * dn2
    dc2 = 1.0 * dn2
    
    # backprop to neuron 3
    da3 = x * dn3
    db3 = y * dn3
    dc3 = 1.0 * dn3
    
    # add pulls from regularization
    da1 += -a1; da2 += -a2; da3 += -a3;
    db1 += -b1; db2 += -b2; db3 += -b3;
    da4 += -a4; db4 += -b4; dc4 += -c4;
    
    # parameter update
    step_size = 0.01
    a1 += step_size * da1
    b1 += step_size * db1 
    c1 += step_size * dc1
    a2 += step_size * da2 
    b2 += step_size * db2
    c2 += step_size * dc2
    a3 += step_size * da3 
    b3 += step_size * db3 
    c3 += step_size * dc3
    a4 += step_size * da4 
    b4 += step_size * db4 
    c4 += step_size * dc4 
    d4 += step_size * dd4

### loss function, 2-D SVM
$L=[\sum_{i=1}^{N}max(0,−y_i(w_0x_{i0}+w_1x_{i1}+w_2)+1)]+\alpha[w_0^2+w_1^2]$

In [96]:
X = [ [1.2, 0.7], [-0.3, 0.5], [3, 2.5] ]
y = [1, -1, 1] # array of labels
w = [0.1, 0.2, 0.3] # example: random weights
alpha = 0.1 # regularization strength

In [97]:
def cost(X,y,w):
    total_cost = 0.0
    N = len(X)
    for i in range(N):
        xi = X[i]
        score = w[0]*xi[0]+w[1]*xi[1]+w[2]
        
        yi = y[i]
        costi = max(0, -yi*score+1)
        print 'example',i,':xi = (',xi,') and label =',yi
        print '  score computed to be ',score
        print '  => cost computed to be ',costi
        total_cost += costi
    
    reg_cost = alpha*(w[0]*w[0]+w[1]*w[1])
    print 'regularization cost for current model is ',reg_cost
    total_cost += reg_cost
    
    print 'total cost is ',total_cost
    return total_cost

In [98]:
total_cost = cost(X,y,w)

example 0 :xi = ( [1.2, 0.7] ) and label = 1
  score computed to be  0.56
  => cost computed to be  0.44
example 1 :xi = ( [-0.3, 0.5] ) and label = -1
  score computed to be  0.37
  => cost computed to be  1.37
example 2 :xi = ( [3, 2.5] ) and label = 1
  score computed to be  1.1
  => cost computed to be  0
regularization cost for current model is  0.005
total cost is  1.815
